# Twisted Probabilities, Uncerntainty, and Prices
by [Lars Peter Hansen](https://larspeterhansen.org/), [Bálint Szőke](https://www.balintszoke.com/), Lloyd S. Han and [Thomas J.Sargent](http://www.tomsargent.com/), you could find the latest draft [here](https://larspeterhansen.org/research/papers/). The notebook presents the code, table and figures accompanying section 6 in the paper

We illustrate effects of concerns about robustness in three environments using a model of capital accumulation with adjustment costs proposed by [Eberly & Wang (2011)](https://www0.gsb.columbia.edu/faculty/nwang/papers/EberlyWang2011_04v3.pdf). We modify their model to expose investment returns  to long-run risks and make investors concerned  about misspecifications of them. Three distinct example economies environments feature:
* a single capital stock
* two capital stocks with a common exposure to long-run risk
*  two capital stocks with only one being exposed to long-run risk

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Baseline-model" data-toc-modified-id="Baseline-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Baseline model</a></span><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Calibration" data-toc-modified-id="Calibration-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Calibration</a></span></li></ul></li><li><span><a href="#Single-Capital-Stock-Case" data-toc-modified-id="Single-Capital-Stock-Case-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Single Capital Stock Case</a></span></li></ul></div>

In [3]:
# Import Necessary packages
using Optim
using Roots
using NPZ

include("newsets_utils.jl")

shock_price (generic function with 1 method)

## Baseline model

### Setup

Aggregate output is proportional to a single capital stock with a constant productivity parameter $\mathcal{A}$. A representative household cares about consumption $C$ with instantaneous utility $\delta\log C$. Under the baseline model, investment $I$ affects capital $K$ according to

\begin{align}
dK_t &= \left[\frac{I_t}{K_t} - \frac{\phi}{2}\left(\frac{I_t}{K_t}\right)^2 + (.01)\left(\widehat \alpha_k + \widehat \beta Z_t \right)\right]K_t dt + (.01)\sigma_k K_t\cdot dW_t \label{eqn:model}\\
dZ_t &= \left(\widehat\alpha_z -\widehat \kappa Z_t \right)dt + \sigma_z \cdot dW_t \nonumber
\tag{1}
\end{align}

with adjustment cost parameter $\phi$. With zero investment, the rate of change of capital is $(.01)\left(\widehat\alpha_k + \widehat\beta Z_t\right)$, where $Z$ is an exogenously specified continuous-time scalar autoregressive process  that puts long-term risks into  returns on capital. Under baseline model $(1)$, the mean of $Z$ is  ${\overline z} = {\frac {{\widehat \alpha}_z} {{\widehat \kappa}}}$.

he aggregate resource constraint is $C + I = {\cal A} K$. It is convenient to pose a robust planner's problem in terms of an investment-capital ratio $D_t \doteq\frac{I_t}{K_t}$ and $\log K_t$ that satisfy
\begin{align*}
d \log K_t = \left[D_t - \frac{\phi}{2}\left(D_t\right)^2 + (.01)\left(\widehat \alpha_k + \widehat \beta Z_t \right) - {\frac {(.01)^2 |\sigma_k|^2}{2}} \right] dt + (.01)\sigma_k \cdot dW_t.
\end{align*}
Denote state variable as $X_t = [\log K_t,\ Z_t]'$ and instantaneous utility function
\begin{align*}
\upsilon(X_t, D_t) = \delta\log\left({\mathcal A}- D_t\right) + \delta\log K_t.
\end{align*}



### Calibration

The basis of our quantitative analysis is an empirical model of  aggregate consumption dynamics. Our code for VAR estimation could be found in __VAR Estimation__ section of Git hub repository [Tenuous Belief](https://github.com/lphansen/TenuousBeliefs).

We follow [Hansen et al. (2008)](http://larspeterhansen.org/wp-content/uploads/2016/10/Consumption-Strikes-Back-Measuring-Long-Run-Risk.pdf) by fitting a trivariate VAR for macroeconomic time series that contain information about long-term consumption growth, namely,
log consumption growth, the difference between logs of business income and consumption, and the difference between logs of personal dividend income and
consumption.

The time series are quarterly data from 1948 Q1 to 2018 Q2. Our consumption measure is per capita consumption of non-durables and services from NIPA Table 1.1.5. Business income is measured as proprietor's income plus corporate profits per capita and the series are from NIPA Table 1.12. Personal dividend income is from NIPA Table 2.1. By including proprietors' income in addition to corporate profits, we use a broader measure of business income than [Hansen et al. (2008)](http://larspeterhansen.org/wp-content/uploads/2016/10/Consumption-Strikes-Back-Measuring-Long-Run-Risk.pdf), who used only corporate profits. Moreover, [Hansen et al. (2008)](http://larspeterhansen.org/wp-content/uploads/2016/10/Consumption-Strikes-Back-Measuring-Long-Run-Risk.pdf) did not include  personal dividends in their VAR analysis.
We restrict  all three time series to have a common martingale component by imposing a known cointegration relation among them.

We convert the discrete time VAR estimates to baseline parameters $({\widehat \alpha}_k,  {\widehat \beta}, {\widehat \alpha}_z, {\widehat \kappa} )$ and $(\sigma_c, \sigma_z)$ by setting ${\widehat \alpha}_z=0$ and $\widehat{\beta}=1$ and matching the dynamics of the VAR implied long-term consumption growth forecast with those of $Z$.
As a result, we obtain the following parameters for the baseline model of consumption $(1)$:

\begin{align} 
& \begin{matrix}
{\widehat \alpha}_c  & = & .484 & & {\widehat \beta} &= &1 \cr
{\widehat \alpha}_z  &= & 0 &  & {\widehat \kappa} & = & .014 \end{matrix} \cr
& \sigma = \begin{bmatrix}
(\sigma_c)' \cr (\sigma_z)' \end{bmatrix}  =  \begin{bmatrix} .477 & 0 \cr  .011 & .025 \end{bmatrix} .
\end{align}

We set the household's subjective discount rate equal to $\delta=.002.$

In more detail, we choose $\widehat{\alpha}_c$ and $(1-\widehat{\kappa})$ to match the VAR implied unconditional mean of consumption growth and autoregressive coefficient of expected long-term consumption growth, respectively. In addition, we set $(\sigma_c, \sigma_z)$ such that $(1, \widehat{\kappa}^{-1})\sigma\sigma'(1, \widehat{\kappa}^{-1})'$ is equal to the VAR implied covariance matrix of the one-step-ahead forecast error for consumption growth and expected long-term consumption growth. We achieve identification by imposing a lower triangular structure with positive diagonal elements on $\sigma$.

In [2]:
#==============================================================================#
#    PARAMETERS
#==============================================================================#
delta = 0.002;

# (1) Baseline model
alpha_z_hat = 0.0;
kappa_hat = 0.014;

alpha_c_hat = 0.484;       # consumption intercept (estimated) 
beta_hat = 1.0;
sigma_c = [0.477, 0.0  ];   # consumption exposure (= exposure of single capital)
sigma_z = [0.011, 0.025];

## Single Capital Stock Case

A planner is concerned about  alternative models for  capital evolution than baseline model $(1)$ in the following parametric class

\begin{align}
d \log K_t &= \left[D_t - \frac{\phi}{2}\left(D_t\right)^2 + (.01)\left( \alpha_k +  \beta Z_t \right) - {\frac {(.01)^2 |\sigma_k|^2}2 } \right] dt + (.01)\sigma_k \cdot dW^H_t \label{eqn:model2} \\
dZ_t &= \left(\alpha_z - \kappa Z_t \right)dt + \sigma_z \cdot dW^H_t \nonumber
\end{align}

where $W^H$ denotes a distorted shock process of the form \eqref{eqn:WWh}. Uncertainty about $(\alpha_k,\beta)$ describes unknown components in  returns to investment while uncertainty about $(\alpha_z,\kappa)$ captures an unknown growth evolution process. We represent a model of the form \eqref{eqn:model2} by restricting the drift distortion $H$ for $W$ to satisfy
\begin{equation}\label{eq:affine_drift}
H_t = \eta(X_t) \doteq {\eta }_0  + {\eta }_1 Z_t
\end{equation}
and using \eqref{eqn:model}  and  \eqref{eqn:model2} to deduce the following restrictions on $\eta_0$ and $\eta_1$ as functions of $(\alpha_k, \beta, \alpha_z,  \kappa)$:
\begin{align}\label{eqn:etarestrictions}
\sigma {\eta}_0  = \begin{bmatrix} {\alpha}_k - {\widehat \alpha}_k \cr {\alpha}_z  -{\widehat \alpha}_z \end{bmatrix}, \hspace{2cm}
\sigma {\eta}_1  = \begin{bmatrix} \beta - {\widehat \beta}  \cr {\widehat \kappa} - { \kappa}  \end{bmatrix}
\end{align}
where
\[
\sigma \doteq \begin{bmatrix} (\sigma_k)' \cr (\sigma_z)' \end{bmatrix} .
\]
Pairs $(\eta_0, \eta_1)$ that satisfy restrictions \eqref{eqn:etarestrictions} represent models having parametric form \eqref{eqn:model2}.

We restrict the set of alternative models by using a quadratic twisting function $\xi(z)$. Following the method of section \ref{sec:structured_example} we ensure that the set $\mathcal{M}(\xi)$ includes a prespecified quartet $({\widetilde \alpha}_k, {\widetilde \beta}, {\widetilde \alpha}_z, {\widetilde \kappa})$ by solving an instance of \eqref{eqn:etarestrictions} for ${\widetilde \eta}_0$ and ${\widetilde \eta}_1$, then setting
\begin{align}\label{eq:xi_general}
\xi(x) =  \left\vert {\widetilde \eta}_0 + {\widetilde \eta}_1 z \right\vert^2.
\end{align}
For the quantitative examples below,  either we twist the pair $\left({\widetilde \alpha}_z, {\widetilde \kappa}\right)$ and form $\xi^{[\kappa]}$, or we twist the pair $({\widetilde \alpha}_z, {\widetilde \beta})$ and form $\xi^{[\beta]}$.


 To set worrisome parameters $\left({\widetilde \alpha}_z, {\widetilde \kappa}\right)$ or $({\widetilde \alpha}_z, {\widetilde \beta})$, we target the entropy of the implied worst-case model $M^{H^*}$ relative to the baseline model \eqref{eqn:model}. As discussed in section \ref{sec:rel_ent}, relative entropy $\Delta^*\left(M^H\right)$ of martingale process $M^H$ equals one half the expectation of $|H_t|^2$ under the $H$ induced probability model. As a result, we can parameterize relative entropy of the worst-case model using the scalar ${\sf q}$:
 \[
 \Delta^*\left(M^{H^*}\right) = {\frac {{\sf q}^2}  2}
\]
One can think of ${\sf q}$ as a measure of the magnitude of a probability distortion. We calibrate worrisome models to induce specific $\mathsf{q}$ values.

To see how state-dependence of a quadratic twisting function influences this calibration, consider again the decomposition $\xi(Z_t)=\xi_0 + \widetilde{\xi}(Z_t)$ and suppose that constraint \eqref{eqn:constraint} is satisfied with equality to arrive at
\begin{align}\label{eq:q_decomposition}
\frac{\mathsf{q}^2}{2} = \Delta^*\left(M^{H^*}\right) = \frac{\xi_0}{2} + \lim_{\delta \downarrow 0} \frac{\delta}{2}\int \exp(-\delta \tau)E\left(M^{H^*}_{\tau} \widetilde{\xi}(Z_{\tau}) \mid   \mathcal F_0\right)d\tau \doteq \xi_0 + \Omega(\xi) .
\end{align}
The last equality defines $\Omega(\xi)$ as the part of relative entropy that arises from state-dependence in $\xi$. Indeed, when \eqref{eq:xi_general} holds, worrisome parameters $\widetilde{\kappa}$ and $\widetilde{\beta}$ influence the implied worst-case model only through $\Omega(\xi)$. For this reason, when we contrast the effects of $\xi^{[\kappa]}$ and $\xi^{[\beta]}$ on the worst-case model in section \ref{sec:quantiative_example}, we will fix $\mathsf{q}$ and $\xi_0$ to make sure that the contributions from state-dependence are the same, i.e.,  $\Omega\left(\xi^{[\kappa]}\right)=\Omega\left(\xi^{[\beta]}\right)$.